<a href="https://colab.research.google.com/github/Dhivyaindhu/project1/blob/main/Expensive_Vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import os
import csv
import re
from typing import Optional, Tuple

class SimpleBillOCR:
    def __init__(self, api_key: str, image_folder: str):
        self.api_key = api_key
        self.image_folder = image_folder
        self.output_file = 'output.csv'

    def extract_text(self, image_path: str) -> Optional[str]:
        """Extract text from image using OCR.space API"""
        try:
            url = 'https://api.ocr.space/parse/image'
            with open(image_path, 'rb') as image_file:
                response = requests.post(
                    url,
                    files={'file': image_file},
                    data={
                        'apikey': self.api_key,
                        'language': 'eng',
                        'detectOrientation': 'true',
                        'scale': 'true',
                        'OCREngine': '2'
                    },
                    timeout=30
                )
            result = response.json()

            if result.get('IsErroredOnProcessing'):
                print(f"OCR Error: {result.get('ErrorMessage', 'Unknown error')}")
                return None

            return result['ParsedResults'][0]['ParsedText']

        except Exception as e:
            print(f"Error processing image: {str(e)}")
            return None

    def clean_amount(self, amount_str: str) -> Optional[float]:
        """Clean and convert amount string to float"""
        try:
            # Remove any non-numeric characters except . and ,
            cleaned = re.sub(r'[^\d.,]', '', amount_str)

            # Handle different decimal separators
            if ',' in cleaned and '.' in cleaned:
                # If both present, assume comma is thousand separator
                cleaned = cleaned.replace(',', '')
            elif ',' in cleaned:
                # If only comma, assume it might be decimal separator
                cleaned = cleaned.replace(',', '.')

            return float(cleaned)
        except ValueError:
            return None

    def is_valid_total(self, amount: float, line: str) -> bool:
        """
        Enhanced validation for total amounts with more flexible rules
        """
        line_lower = line.lower()

        # Ignore lines with bill/invoice number indicators
        number_indicators = [
            r'bill\s*(?:no|number|#)',
            r'invoice\s*(?:no|number|#)',
            r'order\s*(?:no|number|#)',
            r'ref(?:erence)?\s*(?:no|number|#)',
            r'table\s*(?:no|number|#)'
        ]

        if any(re.search(pattern, line_lower) for pattern in number_indicators):
            return False

        # Check for currency symbols
        currency_symbols = ['$', '€', '£', '¥']
        has_currency = any(symbol in line for symbol in currency_symbols)

        # More comprehensive total-related keywords
        total_indicators = [
            'total', 'amount', 'sum', 'paid', 'due', 'payable',
            'grand', 'final', 'net', 'balance', 'subtotal',
            'payment', 'charge', 'price', 'amt', 'bill'
        ]

        # Check for common receipt endings that often precede totals
        end_indicators = [
            'thank you', 'come again', 'total', 'amount',
            'paid', 'due', 'balance', 'cash', 'card'
        ]

        # Look for common receipt patterns
        receipt_patterns = [
            r'cash|card|credit|debit|payment|change|tip',
            r'tax|vat|gst|hst',
            r'thank.*you|come.*again',
            r'server|cashier|operator',
            r'quick\s*service',
            r'drive\s*thru',
            r'restaurant|cafe|diner'
        ]

        context_suggests_total = any(re.search(pattern, line_lower) for pattern in receipt_patterns)

        # If line contains currency symbol and reasonable amount
        if has_currency and 0.01 <= amount <= 1000:
            return True

        # If amount appears in a line with total indicators
        if any(indicator in line_lower for indicator in total_indicators):
            return True

        # If amount appears at the end of receipt with common endings
        if any(indicator in line_lower for indicator in end_indicators):
            return True

        # If amount appears in a total-like context with reasonable value
        if context_suggests_total and 0.01 <= amount <= 1000:
            return True

        # Check if this might be the final total (often standalone number near end)
        is_last_numbers = re.match(r'^\s*[\d.,]+\s*$', line.strip()) is not None
        if is_last_numbers and 0.01 <= amount <= 1000:
            return True

        return False

    def find_total(self, text: str) -> Tuple[Optional[float], Optional[str]]:
        """
        Enhanced total amount detection with more flexible pattern matching
        """
        if not text:
            return None, None

        # Split text into lines and remove empty lines
        lines = [line.strip() for line in text.split('\n') if line.strip()]

        # More flexible patterns to match various receipt formats
        patterns = [
            # Currency symbol patterns (more flexible decimal places)
            (r'[$€£¥]\s*([\d,]+(?:\.\d{0,2})?)', True),

            # Total amount patterns (more flexible)
            (r'(?:total|amount|sum)\s*[:.]?\s*[\$]?\s*([\d,]+(?:\.\d{0,2})?)', True),

            # Explicit total indicators
            (r'(?:grand|final|sub)?\s*total\s*[:.]?\s*([\d,]+(?:\.\d{0,2})?)', True),

            # Amount due/to pay patterns
            (r'(?:amount|sum)\s*(?:due|to\s*pay)\s*[:.]?\s*([\d,]+(?:\.\d{0,2})?)', True),

            # Quick service/restaurant specific patterns
            (r'(?:service|food)\s*(?:charge|amount)\s*[:.]?\s*([\d,]+(?:\.\d{0,2})?)', True),

            # Generic amount pattern (for standalone numbers)
            (r'([\d,]+(?:\.\d{0,2})?)\s*$', False),

            # Simple number pattern as last resort
            (r'([\d,]+)(?!\d)', False)
        ]

        # Store potential matches with their line numbers
        potential_matches = []

        # First pass: Look for amounts with clear total indicators
        for i, line in enumerate(lines):
            for pattern, requires_indicator in patterns:
                matches = re.finditer(pattern, line.lower())

                for match in matches:
                    amount_str = match.group(1)
                    amount = self.clean_amount(amount_str)

                    if amount is not None and self.is_valid_total(amount, line):
                        potential_matches.append((amount, line, i))

        # If we found matches, prefer matches near the end of the receipt
        if potential_matches:
            # Sort by line number (descending) and amount (descending)
            potential_matches.sort(key=lambda x: (-x[2], -x[0]))
            return potential_matches[0][0], potential_matches[0][1]

        # Second pass: Look for standalone numbers near the end
        for line in reversed(lines):
            # Look for any numbers
            numbers = re.findall(r'([\d,]+(?:\.\d{0,2})?)', line)
            for num in reversed(numbers):  # Check numbers from right to left
                amount = self.clean_amount(num)
                if amount is not None and self.is_valid_total(amount, line):
                    return amount, line

        return None, None

    def process_bills(self):
        """Process all bills in the folder and save results to CSV"""
        with open(self.output_file, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['File Name', 'Total Amount', 'Matched Text', 'Full Text'])

            for filename in os.listdir(self.image_folder):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp')):
                    print(f"Processing {filename}...")

                    image_path = os.path.join(self.image_folder, filename)

                    # Extract text using OCR
                    ocr_text = self.extract_text(image_path)
                    if ocr_text is None:
                        writer.writerow([filename, 'Error', 'OCR Failed', ''])
                        continue

                    # Find total amount
                    total_amount, matched_text = self.find_total(ocr_text)

                    # Write results to CSV
                    writer.writerow([
                        filename,
                        f"{total_amount:.2f}" if total_amount is not None else 'Not Found',
                        matched_text or 'Not Found',
                        ocr_text.strip()
                    ])

            print(f"\nProcessing completed. Results saved to {self.output_file}")

def main():
    API_KEY = 'K86357886788957'
    IMAGE_FOLDER = '/content/bills'

    processor = SimpleBillOCR(API_KEY, IMAGE_FOLDER)
    processor.process_bills()

if __name__ == "__main__":
    main()

Processing bill5.jpg...
Processing bill 3.jpg...

Processing completed. Results saved to output.csv
